In [1]:
import os
import pandas as pd

In [19]:
# England - Premier 
# http://www.football-data.co.uk/mmz4281/1819/E0.csv
# Scotland - Premier League
# http://www.football-data.co.uk/mmz4281/1819/SC0.csv
# Germany - Bundesliga
# http://www.football-data.co.uk/mmz4281/1819/D1.csv
# Italy - Serie A
# http://www.football-data.co.uk/mmz4281/1819/I1.csv
# Spain - La Liga
# http://www.football-data.co.uk/mmz4281/1819/SP1.csv
# France - Le championnat
# http://www.football-data.co.uk/mmz4281/1819/F1.csv
leagues = ['E0', 'SC0', 'D1', 'I1', 'SP1', 'F1']

In [20]:
for leag in leagues:
    leag_fn = "data/%s_leag.csv" % leag
    if not os.path.exists(leag_fn):
        print ('Processing league', leag)
        all_matches = pd.DataFrame()
        for season in range(2009, 2018):
            seas = '%02d%02d' % (season-2000, season-1999)
            url = 'http://www.football-data.co.uk/mmz4281/%s/%s.csv' % (seas,leag)
            fn = 'data/%s_%02d.csv' % (leag, season-2000)
            if os.path.exists(fn):
                matches = pd.read_csv(fn, parse_dates=True)
            else:
                print(url)
                matches = pd.read_csv(url, parse_dates=True, error_bad_lines=False, warn_bad_lines=False)
                matches.to_csv(fn, index=False)
            all_matches = pd.concat([all_matches, matches], axis=0, sort=False)
        all_matches = all_matches.dropna(axis=0,how='all')
        # all_matches = all_matches[~all_matches.HS.isna()]
        all_matches = all_matches[~all_matches.B365H.isnull()]
        all_matches = all_matches[~all_matches.AS.isna()]
        all_matches = all_matches.astype({'FTHG':int, 'FTAG':int, 'AS':int, 'HS':int, 'AST':int, 'HST':int
                                 })
        all_matches.to_csv(leag_fn, index=False)

Processing league E0
Processing league SC0
Processing league D1
Processing league I1
Processing league SP1
Processing league F1
http://www.football-data.co.uk/mmz4281/0910/F1.csv
http://www.football-data.co.uk/mmz4281/1011/F1.csv
http://www.football-data.co.uk/mmz4281/1112/F1.csv
http://www.football-data.co.uk/mmz4281/1213/F1.csv
http://www.football-data.co.uk/mmz4281/1314/F1.csv
http://www.football-data.co.uk/mmz4281/1415/F1.csv
http://www.football-data.co.uk/mmz4281/1516/F1.csv
http://www.football-data.co.uk/mmz4281/1617/F1.csv
http://www.football-data.co.uk/mmz4281/1718/F1.csv


In [21]:
import dataset
for leag in leagues:
    leag_fn = "data/%s_leag.csv" % leag
    processed_fn = 'data/%s_processed.csv' % leag
    if os.path.exists(leag_fn) and not os.path.exists(processed_fn):
        print ('Processing league', leag)
        book = dataset.Dataset(leag_fn)
        df = pd.DataFrame(book.processed_results)
        df.to_csv(processed_fn, index=False)

Processing league E0
Processing league SC0
Processing league D1
Processing league I1
Processing league SP1
Processing league F1


In [25]:
dataframes = []
for leag in leagues:
    processed_fn = "data/%s_processed.csv" % leag
    if os.path.exists(processed_fn):
        dataframes.append(pd.read_csv(processed_fn, parse_dates=True))

In [31]:
df = pd.concat(dataframes, ignore_index=True)
df.to_csv('all_processed.csv', index=False)

In [133]:
#nfeatures, labels, odds = utils.get_footballpredictor_feables(csv='temp.csv', use_shots=False)
#nfeatures = nfeatures.iloc[120:,:]
features, labels, odds = utils.get_footballpredictor_feables(csv='temp.csv', use_shots=True)
features = features.iloc[120:,:]
labels = labels.iloc[120:]
odds = odds.iloc[120:,:]

In [125]:
from sklearn.model_selection import StratifiedShuffleSplit
ss = StratifiedShuffleSplit(n_splits=9, random_state=0)

In [126]:
result = pd.DataFrame()
for train_index, test_index in ss.split(features, labels):    
    X_train = features.iloc[train_index]
    X_test = features.iloc[test_index]
    odds_train = odds.iloc[train_index]
    odds_test = odds.iloc[test_index]
    Y_train = labels.iloc[train_index]
    Y_test = labels.iloc[test_index]
    
    y_test = utils.encode_labels(Y_test)
    y_train = utils.encode_labels(Y_train)
    
    x_train = scaler.transform(X_train)
    x_test = scaler.transform(X_test)
    
    b_test = y_test * odds_test - 1
    b_train = y_train * odds_train - 1
    
    r1 = pd.DataFrame(utils.evaluate_model(model, x_train, x_test, b_train, b_test))
    
#     model.fit(x_train, b_train,
#           #epochs=offset+10000,
#           #initial_epoch=offset,
#           epochs=10000,
#           batch_size=500, verbose=0, callbacks=[keras.callbacks.TensorBoard(), 
#                                                 TQDMNotebookCallback(show_inner=False,
#                                                                      #initial=offset                                                                                                    
#                                                                     )
#                                                ])

    r2 = pd.DataFrame(utils.evaluate_model(model, x_train, x_test, b_train, b_test))
    r2.columns = ['trainu', 'testu']
    r = pd.concat([r1, r2], axis=1)
    result = pd.concat([result, r.T['%']], axis=1, sort=False)
    
    clear_output()
    display(result)

,%,%,%,%,%,%,%,%,%
train,-0.046216,-0.038378,-0.043939,-0.029348,-0.040341,-0.037162,-0.024816,-0.036919,-0.032572
test,0.054275,-0.016107,0.033830,-0.097190,0.001517,-0.027022,-0.137886,-0.029206,-0.068244
trainu,-0.046216,-0.038378,-0.043939,-0.029348,-0.040341,-0.037162,-0.024816,-0.036919,-0.032572
testu,0.054275,-0.016107,0.033830,-0.097190,0.001517,-0.027022,-0.137886,-0.029206,-0.068244
